In [ ]:
import numpy as np
import hapke
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from scipy import optimize
from mpl_toolkits import mplot3d
from scipy.interpolate import interp1d
from pyvims import VIMS
from pyvims.misc import MAPS
import random
from pyvims.misc import Map
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patches as patches
from matplotlib.patches import Polygon
from pathlib import Path
from pyvims.wget import wget
plt.rcParams.update({'font.size': 12})

Load cube (using hapke.retrieve_cube) and optical constants (hapke.opticalconstants)

In [ ]:
cube_id = '1741554947_1'

cube = hapke.retrieve_cube(cube_id)

T = 120
N = 10

opt_c = hapke.opticalconstants(T,sensitivity = N)
n_c = opt_c['n']
k_c = opt_c['k']
wav_c = opt_c['wav']

opt_a = hapke.opticalconstants(T, sensitivity= N,crystallinity=False)
n_a = opt_a['n']
k_a = opt_a['k']
wav_a = opt_a['wav']

int_opt = hapke.inter_optical_constants(wav_c, wav_a, n_c, k_c)

wav = np.array(int_opt['wav'])
n_c = int_opt['n']
k_c = int_opt['k']


In [ ]:
bg = MAPS[cube.target_name]
fig, ax = bg.figure(figsize=(20, 10))

ax.add_collection(cube.pixels(2.03, vmin=.05, vmax=.18))
plt.show()

In [ ]:
pixel = cube.get_pixel(60,30)

ax = hapke.plot_pixel_equi(2.03,cube,[(pixel.s,pixel.l)],background=False)

plt.show()

In [ ]:
pixel1 = cube.get_pixel(60,30)
e, i, phase = [np.deg2rad(pixel1.eme),np.deg2rad(pixel1.inc),np.deg2rad(pixel1.phase)]


angles = [e,i,phase]
IF1 = pixel1.spectrum
wav1 = pixel1.wvlns
ini_par = [0.5, 0.00007,0.1]

body = cube.target_name

optimized_parameters = optimize.least_squares(
    hapke.cost_function_mixed_no_weight, ini_par, args=(wav, angles, IF1, wav1,n_c,k_c,n_a,k_a,3.2,body), bounds=([0.001,0.000001,0.001], [0.74,0.001,1], ), x_scale='jac',
)

optimized_values = optimized_parameters.x
hapke.print_error_correlation(optimized_parameters)

print('')
print('Longitude: ' + str(pixel1.lon))
print('Latitude: ' + str(pixel1.lat))
print('Phase: ' + str(pixel1.phase))

This cell below created 20 randomly generated initial parameters and store the results of the fits on a dictionary fit_dict. The variable count adds one every time a set of generated initial parameters leads to the same solution than the previous one.

In [20]:
fit_dict = {}
count = 0
n_fits = 20

for i in range(n_fits):
    param = [random.random()*0.74,random.random()*0.001,random.random()]
    optimized_parameters = optimize.least_squares(
        hapke.cost_function_mixed_no_weight, param, args=(wav, angles, IF1, wav1,n_c,k_c,n_a,k_a,3.2,body), bounds=([0.00001,0.000001,0], [0.74,0.001,1], ),
    )

    fit_dict[i] = {'x': optimized_parameters.x, 'cost': optimized_parameters.cost}

    if i == 0:

        opt_param =  optimized_parameters.x
        opt_cost = optimized_parameters.cost
        count += 1
        print(optimized_parameters.x)

    elif i >= i:

        rel_er1 = abs(opt_param[0] - optimized_parameters.x[0]) / opt_param[0]
        rel_er2 = abs(opt_param[1] - optimized_parameters.x[1]) / opt_param[1]
        rel_er3 =abs(opt_param[2] - optimized_parameters.x[2]) / opt_param[2]

        if rel_er1 > 0.0001 or rel_er2 > 0.0001 or rel_er3 > 0.0001:
            print('PARAMETERS CHANGED')
            best_fit =  optimized_parameters
            print(optimized_parameters.x)
        else:
            count += 1
print(count)

[1.00000000e-05 6.13086131e-05 5.73125082e-01]
20


In [ ]:
print(fit_dict)

In [ ]:
optimized_values = optimized_parameters.x

old_hapke = hapke.hapke_model_mixed(ini_par,wav, angles, n_c,k_c,n_a,k_a,body)['IF']

new_hapke = hapke.hapke_model_mixed(optimized_parameters.x,wav, angles, n_c,k_c,n_a,k_a,body)['IF']

fig, ax = plt.subplots(figsize = (8,3))
ax.plot(wav, old_hapke, label = 'Hapke Initial')
ax.plot(wav, new_hapke, label = 'Hapke Optimized')
ax.plot(wav1, IF1, label = 'Pixel')
ax.set_xlabel('Wavelength [$\mu$m]')
ax.set_ylabel('I/F')
ax.set_xlim(1.1,3.2)
ax.set_title('')
ax.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

plt.imshow(cube@2.03, extent=cube.extent, cmap='gray', vmin=0, vmax=.18)

plt.colorbar(extend='max', label='I/F')

plt.scatter(27, 30, s=150)

plt.xlabel(cube.slabel)
plt.ylabel(cube.llabel)

plt.xticks(cube.sticks)
plt.yticks(cube.lticks)
plt.show()

These coming two cells plot the surface response: the residuals compared to the spectrum taken by VIMS. Note that wa1 and IF1 need to be defined beforehand and that the function hapke_mixed needs to be changes to be 2D in parameters

In [ ]:
# SURFACE RESPONSE OF ENCELADUS SPECTRA

real_par = [0.51,0.00007]

m_range = np.linspace(0.001,1,30)
phi_range = np.linspace(0.001,0.74,30)

residuals = np.zeros((len(phi_range),len(m_range)))


interp_func_2 = interp1d(wav1, IF1, bounds_error=False)

interpolated_lab = interp_func_2(wav)

for i in range(len(phi_range)):
    for j in range(len(m_range)):
        param = [phi_range[i], m_range[j]]
        dif = (interpolated_lab - hapke.hapke_model_mixed(param,wav,angles,n_c,k_c,n_a,k_a,body)['IF'])
        dif = dif[~np.isnan(dif)]
        residuals[i,j] = np.linalg.norm(dif)

In [ ]:
# 3D PLOT OF THE SURFACE RESPONSE

phi_range, m_range = np.meshgrid(phi_range, m_range)
# Define the data

# Create a 3D plot
fig = plt.figure(figsize = (4,4))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(phi_range, m_range, residuals.T, cmap='viridis')
ax.scatter(0.16933,0.502, c = 'green', label = 'Real solution' )
ax.legend()
ax.set_xlabel(r'$\phi$ [-]')
ax.set_ylabel(r'$m$ [-]')
ax.set_zlabel('Residuals')
# Show the plot
plt.show()